In [13]:
# !pip install git+https://github.com/rafiqhasan/auto-tensorflow.git
#!pip install auto-tensorflow
!pip install auto-tensorflow==1.3.4

  Using cached auto_tensorflow-1.3.4-py3-none-any.whl (19 kB)
  Using cached keras_tuner-1.0.4-py3-none-any.whl (97 kB)
INFO: pip is looking at multiple versions of auto-tensorflow to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.6.0 (from auto-tensorflow) (from versions: 2.8.1, 2.8.2, 2.9.0rc0, 2.9.0rc1, 2.9.0, 2.10.0b2, 2.10.0rc0, 2.10.0, 2.11.0rc0, 2.11.0, 2.12.0rc0, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0, 2.14.0rc0, 2.14.0, 2.15.0rc0, 2.15.0)
ERROR: No matching distribution found for tensorflow-text==2.6.0


In [12]:
from auto_tensorflow.tfa import TFAuto

ModuleNotFoundError: No module named 'auto_tensorflow'

### **Download data**

In [3]:
!rm -rf data.*
!rm -rf /content/*.png
!rm -rf *trainer.py
!rm -r /content/train_data
!rm -r /content/test_data
!rm -rf untitled_project
!mkdir /content/train_data
!mkdir /content/test_data
!sudo rm -r /content/tfauto

rm: cannot remove '/content/train_data': No such file or directory
rm: cannot remove '/content/test_data': No such file or directory
rm: cannot remove '/content/tfauto': No such file or directory


In [4]:
# # House price
%%bash
cd /content/train_data
wget https://raw.githubusercontent.com/rafiqhasan/AI_DL_ML_Repo/master/Datasets/house_price/data.csv

cd ../test_data
wget https://raw.githubusercontent.com/rafiqhasan/AI_DL_ML_Repo/master/Datasets/house_price/data.csv ##Taken same data for demonstration purposes only

--2024-02-13 00:17:29--  https://raw.githubusercontent.com/rafiqhasan/AI_DL_ML_Repo/master/Datasets/house_price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1238599 (1.2M) [text/plain]
Saving to: ‘data.csv’

     0K .......... .......... .......... .......... ..........  4% 28.3M 0s
    50K .......... .......... .......... .......... ..........  8% 27.3M 0s
   100K .......... .......... .......... .......... .......... 12% 83.7M 0s
   150K .......... .......... .......... .......... .......... 16% 33.8M 0s
   200K .......... .......... .......... .......... .......... 20%  217M 0s
   250K .......... .......... .......... .......... .......... 24% 73.6M 0s
   300K .......... .......... .......... .......... .......... 28%  201M 0s
   350K ........

In [ ]:
##Initialize TFAuto with root and Data path
tfa = TFAuto(train_data_path='/content/train_data/', test_data_path='/content/test_data/', path_root='/content/tfauto')

In [ ]:
##Step 1
##Run Data setup -> Infer Schema, find anomalies, create profile and show viz
tfa.step_data_explore(viz=False)

In [ ]:
##Step 2
##Run Model Training ->
tfa.step_model_build(label_column = 'price', model_type='REGRESSION')     ##--> Default model_complexity
# tfa.step_model_build(label_column = 'price', model_type='REGRESSION', model_complexity=0)     ##--> Model_complexity = 0 ( Simple model - No HPT )

In [ ]:
##Step 3
##Show model What-If Tool
tfa.step_model_whatif()

In [ ]:
#Check signature
!saved_model_cli show --dir "/content/tfauto/model/1" --all

## **Tensorflow Model Serving**

In [ ]:
!apt-get remove tensorflow-model-server
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

!apt-get install tensorflow-model-server

In [ ]:
###Start Tensorflow server
# %%bash --bg
# export TF_CPP_MIN_VLOG_LEVEL=0

%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8502 \
  --model_name=model \
  --model_base_path="/content/tfauto/model" >server.log 2>&1

In [ ]:
!tail server.log

In [ ]:
import json
import requests

#Create payload
data_py = {"inputs":{'bedrooms': [[3]],
                         'bathrooms': [[2.0]],
                         'sqft_living': [[1180]],
                         'sqft_lot': [[5650]],
                         'floors': [[2.0]],
                         'waterfront': [[1]],
                         'view': [[1]],
                         'condition': [[3]],
                         'grade': [[7]],
                         'sqft_above': [[1180]],
                         'sqft_basement': [[0]],
                         'yr_built': [[1997]],
                         'sqft_living15': [[1340]],
                         'sqft_lot15': [[5650]]
                     }}

data = json.dumps(data_py)
print("payload: ", data)

#Run request on TMS
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8502/v1/models/model:predict', data=data, headers=headers)
json_response.text